In [ ]:
import pandas as pd
from sklearn.externals import joblib
import numpy as np

##### Załadowanie danych

In [ ]:
data = joblib.load('../pickles/events_database.pkl')
fb_raw_full = pd.read_csv('../data/fb_app_db.events_clean_2018.csv')
ss_raw = pd.read_csv('../data/silesiaspace.csv')

##### Podział geolokat z Silesia Space

In [ ]:
ss_raw['latitude'] = ss_raw['geo'].str.rpartition(', ')[0]
ss_raw['longitude'] = ss_raw['geo'].str.rpartition(', ')[2]

##### Dołączenie informacji o liczbie uczestników

In [ ]:
fb_raw_full['facebook_id'] = fb_raw_full['facebook_id'].astype(str)
t = data.merge(fb_raw_full[['facebook_id','attending_count']], how='left')

##### Dołączenie geolokat

In [ ]:
ss_raw['id'] = ss_raw['id'].astype(str)
t = t.merge(ss_raw[['id','latitude','longitude']], left_on = 'id_sk', right_on='id', how='left')
t['lng_std'] = np.where(t['lng_std'].notnull(), t['lng_std'], t['longitude'])
t['lat_std'] = np.where(t['lat_std'].notnull(), t['lat_std'], t['latitude'])
t.drop(columns=['longitude','latitude','id_y'], inplace=True)

##### Zapisanie do pliku

In [ ]:
t.to_csv('../output/events_database.csv')
joblib.dump(t, '../pickles/events_database.pkl')